In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
# 코드 7-7 텐서보드를 사용한 텍스트 분류 모델
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_feature = 2000 #특성으로 사용할 단어의 수
max_len = 500 #사용할 텐스트의 길이

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_feature)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

In [3]:
model = keras.models.Sequential()
model.add(layers.Embedding(max_feature, 128,
                          input_length=max_len,
                          name = "embed"))
model.add(layers.Conv1D(32, 7, activation="relu"))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation="relu"))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d (Conv1D)              (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 98, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
__________________________________________________

In [5]:
model.compile(optimizer="rmsprop",
             loss = "binary_crossentropy",
             metrics = ["acc"])

In [6]:
# 코드 7-8 텐서보드 로그 파일을 위한 디텍터리 생성하기
%mkdir my_log_dir

mkdir: cannot create directory ‘my_log_dir’: File exists


In [7]:
# 코드 7-9 텐서보드 콜백화 함께 모델 훈련하기
callbacks = [
    keras.callbacks.TensorBoard(log_dir="my_log_dir", # 로그 파일이 기록될 위치
                               histogram_freq=1, # 1 에포크마다 활성화 출력의 히스토그램을 기록
                               embeddings_freq=1) # 1 에포크마다 임베딩 데이터를 기록
]

In [8]:
history = model.fit(x_train, y_train,
                   epochs = 20,
                   batch_size = 128,
                   validation_split=0.2,
                   callbacks=callbacks)

Epoch 1/20
157/157 [==============================] - 4s 25ms/step - acc: 0.7032 - loss: 0.5698 - val_loss: 0.4201 - val_acc: 0.8468
Epoch 2/20
157/157 [==============================] - 4s 24ms/step - acc: 0.8585 - loss: 0.4110 - val_loss: 0.5444 - val_acc: 0.8462
Epoch 3/20
157/157 [==============================] - 4s 24ms/step - acc: 0.8832 - loss: 0.3525 - val_loss: 0.4591 - val_acc: 0.8668
Epoch 4/20
157/157 [==============================] - 4s 25ms/step - acc: 0.9021 - loss: 0.3082 - val_loss: 0.5087 - val_acc: 0.8698
Epoch 5/20
157/157 [==============================] - 4s 23ms/step - acc: 0.9225 - loss: 0.2915 - val_loss: 0.4853 - val_acc: 0.8706
Epoch 6/20
157/157 [==============================] - 4s 24ms/step - acc: 0.9424 - loss: 0.2279 - val_loss: 1.1348 - val_acc: 0.7966
Epoch 7/20
157/157 [==============================] - 4s 25ms/step - acc: 0.9527 - loss: 0.1931 - val_loss: 0.7100 - val_acc: 0.8626
Epoch 8/20
157/157 [==============================] - 4s 25ms/step - 

In [14]:
%tensorboard --logdir=my_log_dir

UsageError: Line magic function `%tensorboard` not found.
